In [2]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:

# folder = r"C:\Users\jean-\Desktop\Cours ENSAE\3A ENSAE\ML for trading\Projet\ML-for-trading-project\Données projet\yield curve history Germany"

# # List all files in the folder
# files = os.listdir(folder)

# def extract_year(filename):
#     match = re.search(r'R(\d{2})XX', filename)
#     return match.group(1) if match else None

# # Create the dictionary of DataFrames with desired names
# yields_germany = {
#     f"yields Germany {extract_year(file)} years": pd.read_csv(os.path.join(folder, file),index_col =0)
#     for file in files if extract_year(file) is not None
# }

# list_years = [extract_year(file) for file in files if extract_year(file) is not None]

In [54]:
folder = r"./Données projet/yield curve history Germany"  # Use relative path for GitHub repo

files = os.listdir(folder)

def extract_year(filename):
    match = re.search(r'R(\d{2})XX', filename)
    return match.group(1) if match else None

yields_germany = {
    f"yields Germany {extract_year(file)} years": pd.read_csv(os.path.join(folder, file), index_col=0)
    for file in files if extract_year(file) is not None
}

list_years = [extract_year(file) for file in files if extract_year(file) is not None]

In [55]:

start_date = '2000-01-01'
end_date = '2025-10-13'

business_days = pd.date_range(start=start_date, end=end_date, freq='B').strftime('%Y-%m-%d').tolist()

for year in list_years:
    df = yields_germany[f"yields Germany {year} years"]
    series_filtered = df[df.index.isin(business_days)][f"BBSIS.D.I.ZST.ZI.EUR.S1311.B.A604.R{year}XX.R.A.A._Z._Z.A"]
    df_filtered = pd.DataFrame(series_filtered)
    df_filtered.columns = [f'{year}']
    col_numeric = pd.to_numeric(df_filtered[f'{year}'], errors='coerce')
    df_filtered = pd.DataFrame(col_numeric)
    yields_germany[f"yields Germany {year} years"] = df_filtered[df_filtered[f'{year}'].notnull()]

    
   

In [57]:
df = pd.read_csv('./Données projet/GMD.csv', index_col=0)

In [62]:
df[df.index.isin(['Germany'])].tail(30)

,ISO3,id,year,nGDP,nGDP_USD,rGDP,rGDP_pc,rGDP_USD,deflator,cons,...,cbrate,M0,M1,M2,M3,M4,SovDebtCrisis,CurrencyCrisis,BankingCrisis,CA_USD
countryname,,,,,,,,,,,,,,,,,,,,,
Germany,DEU,DEU,2001,2198333.0,1967162.0,2645544.8,32453.703,2933315.0,83.095665,1662333.0,...,3.0625,83.00000,610266.94,1433600.0,NaN,NaN,0.0,0.0,0.0,-3351.199
Germany,DEU,DEU,2002,2227046.0,2099115.3,2640544.0,32368.137,2927770.3,84.340424,1686651.0,...,2.1250,112.00000,681247.00,1453400.0,NaN,NaN,0.0,0.0,0.0,41715.832
Germany,DEU,DEU,2003,2244321.0,2533791.5,2626177.5,32203.674,2911841.0,85.459610,1722467.0,...,1.1250,126.00000,745839.00,1501799.9,NaN,NaN,0.0,0.0,0.0,38844.754
Germany,DEU,DEU,2004,2285962.0,2840212.5,2644592.3,32466.314,2932258.8,86.439110,1740022.0,...,1.0000,141.00000,782886.94,1515300.0,NaN,NaN,0.0,0.0,0.0,129316.740
Germany,DEU,DEU,2005,2322486.0,2884678.0,2672452.5,32856.645,2963149.5,86.904671,1773661.0,...,1.0625,159.00000,869815.00,1591300.0,NaN,NaN,0.0,0.0,0.0,134347.230
Germany,DEU,DEU,2006,2427038.0,3044575.3,2780481.0,34253.449,3082928.8,87.288422,1818192.0,...,1.9375,176.70500,919149.00,1670100.0,NaN,NaN,0.0,0.0,0.0,177801.380
Germany,DEU,DEU,2007,2545626.0,3483560.8,2863585.8,35356.074,3175073.3,88.896446,1852295.0,...,2.9375,190.09000,960967.94,1849300.0,NaN,NaN,0.0,0.0,0.0,233631.810
Germany,DEU,DEU,2008,2586374.0,3784020.3,2882080.0,35684.984,3195579.3,89.739838,1897355.0,...,2.8125,213.29700,1026569.90,2023500.0,NaN,NaN,0.0,0.0,1.0,212408.940
Germany,DEU,DEU,2009,2492920.0,3467407.0,2723649.0,33841.188,3019914.8,91.528679,1926003.0,...,0.3125,208.41400,1204486.00,1987899.9,NaN,NaN,0.0,0.0,0.0,204409.060
